In [355]:
pd.options.display.max_columns = 100

In [189]:
l1adarkfnames = io.get_current_science_fnames('l1a', 'uvdark', env='production')

/maven_iuvs/production/products/level1a


In [210]:
len(l1adarkfnames)

14108

In [389]:
l1a = io.L1AReader(l1adarkfnames[7010])

In [386]:
l1a.n_unique_spabins

2

In [351]:
l1a.spabins

array([103, 103, 103, 103, 103, 103, 103, 103, 103,  93], dtype=int16)

In [325]:
l1a.img.shape

(10, 256)

In [326]:
l1a.Binning.columns

ColDefs(
    name = 'SPABINWIDTH'; format = '11I'
    name = 'SPABINTRANSMIT'; format = '11I'
    name = 'SPEBINWIDTH'; format = '256I'
    name = 'SPEBINTRANSMIT'; format = '256I'
    name = 'SPAPIXLO'; format = '10I'
    name = 'SPAPIXHI'; format = '10I'
    name = 'SPEPIXLO'; format = '256I'
    name = 'SPEPIXHI'; format = '256I'
    name = 'BINTABLENAME'; format = '8A'
)

In [303]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets

In [387]:
def f(i):
    l1a = io.L1AReader(l1adarkfnames[i])
    print('n_dims:', l1a.n_dims)
    i = 0
    if l1a.n_dims == 3:
        i = 1
    print(l1a.img.shape[i])
    print(l1a.Binning['SPABINWIDTH'].shape[1])

In [388]:
interact(f, i=(0,len(l1adarkfnames)-1));

n_dims: 3
10
12


## Generate dataframe with filename stats

In [191]:
df = pd.DataFrame([io.ScienceFilename(str(i)).as_series() for i in l1adarkfnames])

In [211]:
df.columns

Index(['basename', 'channel', 'cycle_orbit', 'instrument', 'level', 'mission',
       'mode', 'obs_id', 'p', 'phase', 'revision', 'root', 'time', 'timestr',
       'tokens', 'version', 'version_string'],
      dtype='object')

In [193]:
df['phase'].value_counts()

IPH2            3181
apoapse         3123
periapse        1863
early           1754
outbound         785
IPH1             672
inbound          405
outboundhifi     372
ISON2            299
IPH3             226
ISON1            224
occultation      217
centroid         208
cruisecal2       146
outdisk          142
outlimb          134
APP1             112
APP1A             56
APP2              55
cruisecal1        50
outdiskhifi       36
comet             22
periapsehifi      20
checkout           4
                   2
Name: phase, dtype: int64

In [368]:
from IPython.parallel import Client
c = Client()

In [369]:
dview = c.direct_view()
lview = c.load_balanced_view()

## When I only scan for darks, len(fnames) < 10000, so I can keep the dataframes in memory.

### If that is a problem, don't return the dataframe at the end!

In [370]:
def check_for_issues(p):
    from iuvs import exceptions
    d = {}
    d['fname'] = p.name
    try:
        l1a = io.L1AReader(str(p))
    except exceptions.DimensionsError:
        d['dims'] = False
    if l1a.img_header['BIN_TBL'].startswith('NON LINEAR'):
        d['bintbl'] = 'nonlinear'
    try:
        _ = l1a.img_header['SPA_SIZE']
    except KeyError:
        d['spasize'] = False
    if not l1a.img_header['FILENAME'].endswith('.fits.gz'):
        d['filename'] = False
    d['lenspabins'] = len(set(l1a.spabins))
    d['lenspebins'] = len(set(l1a.spebins))
    return d

In [371]:
check_for_issues(df.p[4])

{'filename': False,
 'fname': 'mvn_iuv_l1a_APP1-orbit00084-mode0111-muvdark_20141013T232042_v02_r01.fits.gz',
 'lenspabins': 2,
 'lenspebins': 1}

In [383]:
doing = df.p
results = lview.map_async(check_for_issues, doing)

In [394]:
from iuvs.multitools import nb_progress_display
results.ready()

True

In [384]:
nb_progress_display(results, doing)

KeyboardInterrupt: 

In [395]:
resultdf = pd.DataFrame(results.result)

In [396]:
for col in resultdf.columns:
    if col == 'fname': continue
    print(col)
    print(resultdf[col].value_counts(dropna=False))

bintbl
NaN          13734
nonlinear      374
Name: bintbl, dtype: int64
filename
False    14108
Name: filename, dtype: int64
lenspabins
1    13543
2      565
Name: lenspabins, dtype: int64
lenspebins
1    13147
2      775
9      186
Name: lenspebins, dtype: int64
spasize
NaN      13734
False      374
Name: spasize, dtype: int64


In [377]:
resultdf

bintbl filename                                              fname  \
0    nonlinear    False  mvn_iuv_l1a_-orbit01580-fuvdark_20150722T16250...   
1    nonlinear    False  mvn_iuv_l1a_-orbit01655-fuvdark_20150805T07574...   
2          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0111-fuvdark_2...   
3          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0111-fuvdark_2...   
4          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0111-muvdark_2...   
5          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0111-muvdark_2...   
6          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0112-fuvdark_2...   
7          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0112-fuvdark_2...   
8          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0112-muvdark_2...   
9          NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0112-muvdark_2...   
10         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0211-fuvdark_2...   
11         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0211-fuvdark_2...   
12         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0211-muvdark_2...   
13         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0211-muvdark_2...   
14         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0212-fuvdark_2...   
15         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0212-fuvdark_2...   
16         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0212-muvdark_2...   
17         NaN    False  mvn_iuv_l1a_APP1-orbit00084-mode0212-muvdark_2...   
18         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0311-fuvdark_2...   
19         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0311-fuvdark_2...   
20         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0311-muvdark_2...   
21         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0311-muvdark_2...   
22         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0312-fuvdark_2...   
23         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0312-fuvdark_2...   
24         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0312-muvdark_2...   
25         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0312-muvdark_2...   
26         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0411-fuvdark_2...   
27         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0411-fuvdark_2...   
28         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0411-muvdark_2...   
29         NaN    False  mvn_iuv_l1a_APP1-orbit00085-mode0411-muvdark_2...   
..         ...      ...                                                ...   
130        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0311-fuvdark_...   
131        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0311-fuvdark_...   
132        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0311-muvdark_...   
133        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0311-muvdark_...   
134        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0312-fuvdark_...   
135        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0312-fuvdark_...   
136        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0312-muvdark_...   
137        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0312-muvdark_...   
138        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0411-fuvdark_...   
139        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0411-fuvdark_...   
140        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0411-muvdark_...   
141        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0411-muvdark_...   
142        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0412-fuvdark_...   
143        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0412-fuvdark_...   
144        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0412-muvdark_...   
145        NaN    False  mvn_iuv_l1a_APP1A-orbit00168-mode0412-muvdark_...   
146        NaN    False  mvn_iuv_l1a_APP1A-orbit00169-mode0511-fuvdark_...   
147        NaN    False  mvn_iuv_l1a_APP1A-orbit00169-mode0511-fuvdark_...   
148        NaN    False  mvn_iuv_l1a_APP1A-orbit00169-mode0511-muvdark_...   
149        NaN    False  mvn_i

In [209]:
resultdf[resultdf[1]== 1][0].to_csv('/home/klay6683/to_keep/missing_binning_fields.csv')

In [206]:
resultdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14108 entries, 0 to 14107
Data columns (total 2 columns):
0    14108 non-null object
1    14108 non-null int64
dtypes: int64(1), object(1)
memory usage: 330.7+ KB


In [392]:
l1a.primary_img_dn_s.mean(axis=(1,2))

array([ 0.28499987,  0.29065173,  0.28991705,  0.29504912,  0.28251009,
        0.29542302,  0.30167664,  0.28757104,  0.2780717 ,  0.27904405,
        0.29417564,  0.31145206,  0.28650963,  0.29835663,  0.28400402,
        0.29205689,  0.29192985,  0.2860661 ,  0.30517855,  0.28293447,
        0.28134726])

In [19]:
def process_fname(fname):
    import numpy as np
    l1a = io.L1AReader(fname)
    d = dict(fname=fname)
    try:
        l1a = io.L1AReader(str(p))
    except exceptions.DimensionsError:
        d['error'] = 'dims'
        return d
    if l1a.img_header['BIN_TBL'].startswith('NON LINEAR'):
        d['error'] = 'nonlinear'
        return d
    d['n_unique_spa_bins'] = l1a.n_unique_spabins
    d['n_unique_spe_bins'] = l1a.n_unique_spebins
    main_header = io.get_header_df(l1a.hdulist[0])
    integration = l1a.Integration
    if main_header['NAXIS'] == 2:
        main_header['NAXIS3'] = np.nan
        avgtuple = None
    elif main_header['NAXIS'] == 3:
        avgtuple = (1,2)
    else:
        d['axes'] = False
        return d
    try:
        integration['mean'] = l1a.primary_img_dn_s.mean(axis=avgtuple)
        integration['median'] = np.median(l1a.primary_img_dn_s, axis=avgtuple)
        integration['std'] = l1a.primary_img_dn_s.std(axis=avgtuple)
    except KeyError:
        integration['mean'] = np.nan
        integration['median'] = np.nan
        integration['std'] = np.nan
    joined = integration.join(pd.concat([pd.DataFrame(main_header).T] * len(integration),
                                        ignore_index=True))
    for col in l1a.Observation.names[:-3]:
        val = l1a.Observation[col][0]
        if col == 'COLLECTION_ID':
            val = val[0]
        joined[col] = val
    joined = joined.convert_objects(convert_numeric=True)
#     savepath = io.save_to_hdf(joined, fname, 'l1a_dark_scans')
#     return joined, 0

In [24]:
process_fname(fnames[1])

ValueError: Length of values does not match length of index

In [20]:
results = lview.map_async(process_fname, fnames[:8])

In [7]:
from iuvs.multitools import nb_progress_display

In [21]:
nb_progress_display(results, fnames)

In [22]:
naxis_not_2_or_3 = []
good_ones = []
n_ints_larger1 = []
for res in results:
    if res[1] == 1:
        naxis_not_2_or_3.append(res[0])
    elif res[1] == 2:
        n_ints_larger1.append(res[0])
    else:
        good_ones.append(res[0])

CompositeError: one or more exceptions from call to method: process_fname
[4:apply]: ValueError: Length of values does not match length of index
[0:apply]: ValueError: Length of values does not match length of index
[1:apply]: ValueError: Length of values does not match length of index

In [45]:
len(n_ints_larger1)

6

In [46]:
len(naxis_not_2_or_3)

5

In [47]:
n_ints_larger1

['/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-muvdark_20150521T095658_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01236-muvdark_20150520T035347_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01236-fuvdark_20150520T035347_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-fuvdark_20150521T095658_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-fuvdark_20150521T104548_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-fuvdark_20150521T120507_v02_s01.fits.gz']

In [59]:
results_df = pd.concat(good_ones, ignore_index=True)

In [60]:
results_df.head()

ANC_SVN_REVISION   BIN_PATTERN_INDEX                   BIN_TBL  BITPIX  \
0              929  LINEAR linear_0004    LINEAR 5,6 linear_0004      32   
1              929  LINEAR linear_0006    LINEAR 5,6 linear_0006      32   
2              929  LINEAR linear_0006    LINEAR 5,6 linear_0006      32   
3              929  LINEAR linear_0004  LINEAR 15,16 linear_0004      32   
4              929       LINEAR CSS_S4       LINEAR 17,18 CSS_S4      32   

   BLANK BUNDLE_ID  CADENCE                            CAPTURE  CASE_TEMP  \
0     -1       raw      4.8  2014/338 Dec 04 02:03:22.79468UTC      15972   
1     -1       raw      4.8  2015/008 Jan 08 18:21:38.42850UTC      15964   
2     -1       raw      4.8  2015/008 Jan 08 18:21:38.42850UTC      15964   
3     -1       raw     15.0  2014/336 Dec 02 05:40:46.04215UTC      15928   
4     -1       raw     15.0  2014/291 Oct 18 17:54:10.74999UTC      15929   

  CHANNEL CODE_SVN_REVISION COLLECTION_ID  DET_TEMP  DUTY_CYCLE            ET  \
0     FUV               929          limb      2594    1.000000  4.709307e+08   
1     FUV               929          limb      2600    1.000000  4.740134e+08   
2     FUV               929          limb      2604    1.000000  4.740134e+08   
3     FUV               929          disk      2589    1.000000  4.707709e+08   
4     MUV          929:931M          disk      2572    0.200855  4.669269e+08   

  EXTEND                                           FILENAME    FOV_DEG  \
0   True  mvn_iuv_l1a_periapse-orbit00351-fuvdark_201412...  23.373413   
1   True  mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...  17.292480   
2   True  mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...  17.292480   
3   True  mvn_iuv_l1a_apoapse-orbit00341-fuvdark_2014120...  65.797119   
4   True  mvn_iuv_l1a_apoapse-orbit00109-muvdark_2014101...  65.797119   

  GRATING_SELECT  INT_TIME KEYHOLE_SELECT  LYA_CENTROID  MCP_VOLT  MIRROR_DEG  \
0         LOWRES       4.2        NEITHER             1     -1.83   11.686707   
1         LOWRES       4.2        NEITHER             1     -1.83    8.646240   
2         LOWRES       4.2        NEITHER             1     -1.83    8.646240   
3         LOWRES      14.4        NEITHER             1     -1.83   32.898560   
4         LOWRES      14.4        NEITHER            -1     -1.83   32.898560   

   MIRROR_DN  MIR_DEG MISSION_PHASE  NAXIS  NAXIS1  NAXIS2  NAXIS3  N_FILL  \
0      17194   11.689         PRIME      2     256       7     NaN       0   
1      16087    8.649         PRIME      3     256       7       2       0   
2      16087    8.649         PRIME      3     256       7       2       0   
3      24917   32.901         PRIME      2     184      10     NaN       0   
4      24917   32.901         PRIME      2      30      10     NaN       0   

  OBSERVATION_TYPE  OBS_ID  ORBIT_NUMBER  ORBIT_SEGMENT  \
0              CDS       4           351              0   
1              CDS       6           538              0   
2              CDS       6           538              0   
3              CDS    8196           341              0   
4              CDS    2001           109              0   

                             PROCESS              PRODUCT_CREATION_DATE  \
0  2015/108 Apr 18 18:28:06.00000UTC  2015/108 Apr 18 18:28:06.00000UTC   
1  2015/108 Apr 18 22:22:19.00000UTC  2015/108 Apr 18 22:22:19.00000UTC   
2  2015/108 Apr 18 22:22:19.00000UTC  2015/108 Apr 18 22:22:19.00000UTC   
3  2015/108 Apr 18 18:13:25.00000UTC  2015/108 Apr 18 18:13:25.00000UTC   
4  2015/115 Apr 25 03:58:15.00000UTC  2015/115 Apr 25 03:58:15.00000UTC   

                                          PRODUCT_ID SIMPLE  SOLAR_LONGITUDE  \
0  mvn_iuv_l1a_periapse-orbit00351-fuvdark_201412...   True       245.792343   
1  mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...   True       268.399689   
2  mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...   True       268.399689   
3  mvn_iuv_l1a_apoapse-orbit00341-fuvdark_2014120...   True       24

In [61]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18826 entries, 0 to 18825
Data columns (total 53 columns):
ANC_SVN_REVISION         18826 non-null object
BIN_PATTERN_INDEX        18826 non-null object
BIN_TBL                  18826 non-null object
BITPIX                   18826 non-null int64
BLANK                    18826 non-null int64
BUNDLE_ID                18826 non-null object
CADENCE                  18826 non-null float64
CAPTURE                  18826 non-null object
CASE_TEMP                18826 non-null float64
CHANNEL                  18826 non-null object
CODE_SVN_REVISION        18826 non-null object
COLLECTION_ID            18826 non-null object
DET_TEMP                 18826 non-null float64
DUTY_CYCLE               18826 non-null float64
ET                       18826 non-null float64
EXTEND                   18826 non-null bool
FILENAME                 18826 non-null object
FOV_DEG                  18826 non-null float64
GRATING_SELECT           18826 non-null obj

In [62]:
results_df.loc[results_df.ANC_SVN_REVISION == '', 'ANC_SVN_REVISION'] = 0

In [63]:
results_df = results_df.convert_objects(convert_numeric=True)

In [64]:
results_df.to_hdf('/home/klay6683/output/l1a_dark_scans/results_df.h5', 'df')

### For longer scans when I can't keep everything in memory

In [10]:
import glob
h5fnames = glob.glob("/home/klay6683/output/l1a_dark_scans/*.h5")

In [11]:
len(h5fnames)

40421

In [12]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [13]:
dfs = []
for i,chunk in enumerate(chunker(h5fnames, 200)):
    print("Chunk {}".format(i))
    frames = []
    for fname in chunk:
        frames.append(pd.read_hdf(fname, 'df'))
    dfs.append(pd.concat(frames, ignore_index=True))

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Chunk 48
Chunk 49
Chunk 50
Chunk 51
Chunk 52
Chunk 53
Chunk 54
Chunk 55
Chunk 56
Chunk 57
Chunk 58
Chunk 59
Chunk 60
Chunk 61
Chunk 62
Chunk 63
Chunk 64
Chunk 65
Chunk 66
Chunk 67
Chunk 68
Chunk 69
Chunk 70
Chunk 71
Chunk 72
Chunk 73
Chunk 74
Chunk 75
Chunk 76
Chunk 77
Chunk 78
Chunk 79
Chunk 80
Chunk 81
Chunk 82
Chunk 83
Chunk 84
Chunk 85
Chunk 86
Chunk 87
Chunk 88
Chunk 89
Chunk 90
Chunk 91
Chunk 92
Chunk 93
Chunk 94
Chunk 95
Chunk 96
Chunk 97
Chunk 98
Chunk 99
Chunk 100
Chunk 101
Chunk 102
Chunk 103
Chunk 104
Chunk 105
Chunk 106
Chunk 107
Chunk 108
Chunk 109
Chunk 110


In [14]:
superdf = pd.concat(dfs, ignore_index=True)

In [15]:
superdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40421 entries, 0 to 40420
Data columns (total 63 columns):
AVERAGE               40421 non-null float64
BIN_SHIFT             40421 non-null int64
BIN_SHIFT_DIR         40421 non-null int64
BIN_TBL               40421 non-null object
BIN_TYPE              40421 non-null object
BIN_X_ROW             40421 non-null int64
BIN_Y_ROW             40421 non-null int64
BITPIX                40421 non-null int64
BLANK                 40421 non-null int64
BSCALE                494 non-null float64
BZERO                 494 non-null float64
CADENCE               40421 non-null float64
CAPTURE               40421 non-null object
CASE_TEMP             39927 non-null float64
CHECKSUM              40421 non-null float64
DATA_COMPRESSION      40421 non-null int64
DET_TEMP              39927 non-null float64
EXTEND                40421 non-null bool
FILENAME              40421 non-null object
FUV_BAD_PIXEL_MASK    40421 non-null int64
IMAGE_NUMBER      

In [18]:
from iuvs import calib

In [19]:
superdf.DET_TEMP = superdf.DET_TEMP.map(calib.convert_det_temp_to_C)
superdf.CASE_TEMP = superdf.CASE_TEMP.map(calib.convert_case_temp_to_C)

In [20]:
superdf.to_hdf('/home/klay6683/output/l1a_summary.h5','df')

In [21]:
meta.produce_summary_txt(superdf, 'l1a_summary.txt')